## Punto 4 - Fine Tunning

In [148]:
do_fine_tuning = False

lang = "english"

#HPC calculated files
input_model_file = f"data_hpc/{lang}/model_pre_tunning_{lang}.pkl"
input_train_dataset_file = f"data_hpc/{lang}/train_dataset_{lang}.pkl"
output_model_file = f"data_hpc/{lang}/model_post_tunning_{lang}.pkl"

#Models path
model_folder_path = f'models/{lang}/qna_english_custom'

#Dataset
dataset_path = "covid_qa_deepset"

Importar dataset

In [149]:
from datasets import load_dataset

dataset = load_dataset("covid_qa_deepset")
print(type(dataset))

<class 'datasets.dataset_dict.DatasetDict'>


In [150]:
num_docs = print(len(dataset['train']))

2019


In [151]:
print(dataset['train'][0])
ejemplo = dataset['train'][0]

{'answers': {'answer_start': [370], 'text': ['Mother-to-child transmission (MTCT) is the main cause of HIV-1 infection in children worldwide.']}, 'context': "Functional Genetic Variants in DC-SIGNR Are Associated with Mother-to-Child Transmission of HIV-1\n\nhttps://www.ncbi.nlm.nih.gov/pmc/articles/PMC2752805/\n\nBoily-Larouche, Geneviève; Iscache, Anne-Laure; Zijenah, Lynn S.; Humphrey, Jean H.; Mouland, Andrew J.; Ward, Brian J.; Roger, Michel\n2009-10-07\nDOI:10.1371/journal.pone.0007211\nLicense:cc-by\n\nAbstract: BACKGROUND: Mother-to-child transmission (MTCT) is the main cause of HIV-1 infection in children worldwide. Given that the C-type lectin receptor, dendritic cell-specific ICAM-grabbing non-integrin-related (DC-SIGNR, also known as CD209L or liver/lymph node–specific ICAM-grabbing non-integrin (L-SIGN)), can interact with pathogens including HIV-1 and is expressed at the maternal-fetal interface, we hypothesized that it could influence MTCT of HIV-1. METHODS AND FINDINGS:

In [152]:
print(type(dataset['train']))


<class 'datasets.arrow_dataset.Dataset'>


In [153]:
dataset_data = dataset['train']
dataset_data = dataset_data.select(range(100))

dataset_data

Dataset({
    features: ['document_id', 'context', 'question', 'is_impossible', 'id', 'answers'],
    num_rows: 100
})

In [154]:
dataset_completo = dataset_data.train_test_split(0.4, seed=1)
data_train = dataset_completo["train"]
data_test_val = dataset_completo["test"]

In [155]:
dataset_completo = data_test_val.train_test_split(0.5)
data_test = dataset_completo["train"]
data_val = dataset_completo["test"]


In [156]:
def read_dataframe(dataset):
    contexts = []
    questions = []
    answers = []
    dataset.map(lambda example: contexts.append(example['context']))
    dataset.map(lambda example: questions.append(example['question']))
    dataset.map(lambda example: answers.append(example['answers']))
    return contexts, questions, answers

In [157]:
train_contexts, train_questions, train_answers = read_dataframe(data_train)
val_contexts, val_questions, val_answers = read_dataframe(data_val)
test_contexts, test_questions, test_answers = read_dataframe(data_test)

In [158]:
print(len(train_contexts))
print(len(val_contexts))
print(len(test_contexts))

60
20
20


In [159]:
'''
El formato tenía listas en vez de string e ints
'''
def ajustar_answers(answers):
    for answer in answers:
         answer['text'] = answer['text'][0]
         answer['answer_start'] = int(answer['answer_start'][0])
    return answers

train_answers = ajustar_answers(train_answers)
val_answers = ajustar_answers(val_answers)
test_answers = ajustar_answers(test_answers)

In [160]:
def add_end_idx(answers, contexts):
    count = 0
    #print("entra1")
    # loop through each answer-context pair
    print(len(answers))
    for answer, context in zip(answers, contexts):
        # gold_text refers to the answer we are expecting to find in context
        gold_text = answer['text']
        #print(gold_text)
        # we already know the start index
        start_idx = answer['answer_start']
        # and ideally this would be the end index...
        end_idx = start_idx + len(gold_text)
        #print(start_idx)
        #print(end_idx)
        #print("context", context[start_idx:end_idx])
        # ...however, sometimes squad answers are off by a character or two
        if context[start_idx:end_idx] == gold_text:
            # if the answer is not off :)
            answer['answer_end'] = end_idx
            #print("entra2")
            count = count+1

        else:
            #print("entra3")
            for n in range(4):
                #print(n)
                problema = True
                if gold_text == context[start_idx+n:end_idx+n]:
                    # this means the answer is off by 'n' tokens
                    answer['answer_start'] = start_idx + n
                    answer['answer_end'] = end_idx + n
                    #print("entra4")
                    count = count+1
                    problema = False
                    break
                if gold_text == context[start_idx-n:end_idx-n]:
                    answer['answer_start'] = start_idx - n
                    answer['answer_end'] = end_idx - n
                    count = count+1
                    problema = False
                    break
            if problema:
                print("1",gold_text)
                print("2",context[start_idx+n:end_idx+n])
            #holi = holi and context[start_idx+1:end_idx+1] == gold_text
            #print(context[start_idx+1:end_idx+1] == gold_text)
            #break
    print(count)

add_end_idx(train_answers, train_contexts)
add_end_idx(test_answers, test_contexts)
add_end_idx(val_answers, val_contexts)

60
60
20
20
20
20


In [161]:
from transformers import BertForQuestionAnswering, AutoTokenizer

modelname = 'deepset/bert-base-cased-squad2'

model = BertForQuestionAnswering.from_pretrained(modelname)
tokenizer = AutoTokenizer.from_pretrained(modelname)

In [162]:
train_encodings = tokenizer(train_contexts, train_questions, truncation=True, padding=True)
val_encodings = tokenizer(val_contexts, val_questions, truncation=True, padding=True)

In [163]:
def add_token_positions(encodings, answers):
    # initialize lists to contain the token indices of answer start/end
    start_positions = []
    end_positions = []
    for i in range(len(answers)):
        # append start/end token position using char_to_token method
        start_positions.append(encodings.char_to_token(i, answers[i]['answer_start']))
        end_positions.append(encodings.char_to_token(i, answers[i]['answer_end']))

        # if start position is None, the answer passage has been truncated
        if start_positions[-1] is None:
            start_positions[-1] = tokenizer.model_max_length
        # end position cannot be found, char_to_token found space, so shift one token forward
        go_back = 1
        while end_positions[-1] is None:
            end_positions[-1] = encodings.char_to_token(i, answers[i]['answer_end']-go_back)
            go_back +=1
    # update our encodings object with the new token-based start/end positions
    encodings.update({'start_positions': start_positions, 'end_positions': end_positions})

# apply function to our data
add_token_positions(train_encodings, train_answers)
add_token_positions(val_encodings, val_answers)

In [164]:
import torch

class SquadDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)

train_dataset = SquadDataset(train_encodings)
val_dataset = SquadDataset(val_encodings)

In [165]:
import pickle as pk

with open(input_model_file, "wb") as file:
    pk.dump(model, file)

with open(input_train_dataset_file, "wb") as file:
    pk.dump(train_dataset, file)

In [166]:
'''
Excecuted with HPC
'''
if do_fine_tuning:
    from torch.utils.data import DataLoader
    from transformers import AdamW
    from tqdm import tqdm

    # setup GPU/CPU
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    # move model over to detected device
    model.to(device)
    # activate training mode of model
    model.train()
    # initialize adam optimizer with weight decay (reduces chance of overfitting)
    optim = AdamW(model.parameters(), lr=5e-5)

    # initialize data loader for training data
    train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

    for epoch in range(3):
        # set model to train mode
        model.train()
        # setup loop (we use tqdm for the progress bar)
        loop = tqdm(train_loader, leave=True)
        for batch in loop:
            # initialize calculated gradients (from prev step)
            optim.zero_grad()
            # pull all the tensor batches required for training
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            start_positions = batch['start_positions'].to(device)
            end_positions = batch['end_positions'].to(device)
            # train model on batch and return outputs (incl. loss)
            outputs = model(input_ids, attention_mask=attention_mask,
                            start_positions=start_positions,
                            end_positions=end_positions)
            # extract loss
            loss = outputs[0]
            # calculate loss for every parameter that needs grad update
            loss.backward()
            # update parameters
            optim.step()
            # print relevant info to progress bar
            loop.set_description(f'Epoch {epoch}')
            loop.set_postfix(loss=loss.item())
else:
    with open(output_model_file, "rb") as file:
        model = pk.load(file)

FileNotFoundError: [Errno 2] No such file or directory: 'data_hpc/english/model_post_tunning_english'

In [ ]:
model.save_pretrained(model_folder_path)
tokenizer.save_pretrained(model_folder_path)

In [167]:
model2 = BertForQuestionAnswering.from_pretrained(model_folder_path)
tokenizer2 = AutoTokenizer.from_pretrained(model_folder_path)

## Creación del pipeline

In [168]:
from transformers import pipeline
nlp = pipeline('question-answering', model=model2, tokenizer=tokenizer2)


## Pruebas en inglés


In [169]:
context = "Masks should be used as part of a comprehensive strategy of measures to suppress transmission and save lives; the use of a mask alone is not sufficient to provide an adequate level of protection against COVID-19. If COVID-19 is spreading in your community, stay safe by taking some simple precautions, such as physical distancing, wearing a mask, keeping rooms well ventilated, avoiding crowds, cleaning your hands, and coughing into a bent elbow or tissue. Check local advice where you live and work. Do it all! Make wearing a mask a normal part of being around other people. The appropriate use, storage and cleaning or disposal of masks are essential to make them as effective as possible."

nlp({
    'question': 'What precautions can you take to stay safe?',
    'context': context
})

{'score': 0.02689635194838047,
 'start': 310,
 'end': 345,
 'answer': 'physical distancing, wearing a mask'}

In [170]:
# Taken from: https://www.who.int/emergencies/diseases/novel-coronavirus-2019/covid-19-vaccines/advice

context = "The world is in the midst of a COVID-19 pandemic. As WHO and partners work together on the response -- tracking the pandemic, advising on critical interventions, distributing vital medical supplies to those in need--- they are racing to develop and deploy safe and effective vaccines. Vaccines save millions of lives each year. Vaccines work by training and preparing the body’s natural defences – the immune system – to recognize and fight off the viruses and bacteria they target. After vaccination, if the body is later exposed to those disease-causing germs, the body is immediately ready to destroy them, preventing illness. There are several safe and effective vaccines that prevent people from getting seriously ill or dying from COVID-19. This is one part of managing COVID-19, in addition to the main preventive measures of staying at least 1 metre away from others, covering a cough or sneeze in your elbow, frequently cleaning your hands, wearing a mask and avoiding poorly ventilated rooms or opening a window."

test1 = nlp({
    'question': 'What happens after vaccination?',
    'context': context
})

print(test1)

{'score': 0.292270690202713, 'start': 563, 'end': 628, 'answer': 'the body is immediately ready to destroy them, preventing illness'}


## Metricas

In [171]:
#Basado en: https://qa.fastforwardlabs.com/no%20answer/null%20threshold/bert/distilbert/exact%20match/f1/robust%20predictions/2020/06/09/Evaluating_BERT_on_SQuAD.html

def get_prediction(qid, questions, contexts):
    specific_question = questions[qid]
    specific_context = contexts[qid]

    answer = nlp({
    'question': specific_question,
    'context': specific_context
    })
    #print(answer)
    return answer["answer"]

#holi = get_prediction(0, test_questions, test_contexts)
#print(holi)

In [172]:
#Basado en: https://qa.fastforwardlabs.com/no%20answer/null%20threshold/bert/distilbert/exact%20match/f1/robust%20predictions/2020/06/09/Evaluating_BERT_on_SQuAD.html

def normalize_text(s):
    """Removing articles and punctuation, and standardizing whitespace are all typical text processing steps."""
    import string, re

    def remove_articles(text):
        regex = re.compile(r"\b(a|an|the)\b", re.UNICODE)
        return re.sub(regex, " ", text)

    def white_space_fix(text):
        return " ".join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return "".join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))

def compute_exact_match(prediction, truth):
    return int(normalize_text(prediction) == normalize_text(truth))

def compute_f1(prediction, truth):
    pred_tokens = normalize_text(prediction).split()
    truth_tokens = normalize_text(truth).split()

    # if either the prediction or the truth is no-answer then f1 = 1 if they agree, 0 otherwise
    if len(pred_tokens) == 0 or len(truth_tokens) == 0:
        return int(pred_tokens == truth_tokens)

    common_tokens = set(pred_tokens) & set(truth_tokens)

    # if there are no common tokens then f1 = 0
    if len(common_tokens) == 0:
        return 0

    prec = len(common_tokens) / len(pred_tokens)
    rec = len(common_tokens) / len(truth_tokens)

    return 2 * (prec * rec) / (prec + rec)

def get_gold_answers(example):
    """helper function that retrieves all possible true answers from a squad2.0 example"""

    gold_answers = [answer["text"] for answer in example.answers if answer["text"]]

    # if gold_answers doesn't exist it's because this is a negative example -
    # the only correct answer is an empty string
    if not gold_answers:
        gold_answers = [""]

    return gold_answers

In [173]:
n = 1
prediction = get_prediction(n, questions=test_questions, contexts=test_contexts)
#print(prediction)
answer = test_answers[n]["text"]
#print(answer)

em_score = compute_exact_match(prediction, answer)
f1_score = compute_f1(prediction, answer)

print(f"Question: {test_questions[n]}")
print(f"Prediction: {prediction}")
print(f"Answer: {answer}")
print(f"EM: {em_score} \t F1: {f1_score}")

Question: What is another name for IFITM5?
Prediction: bone formation
Answer: bonerestricted IFITM-like (BRIL) protein
EM: 0 	 F1: 0


In [174]:
'''
Demorado
'''
def average_metrics(contexts, answers, questions):
    if len(contexts)!= len(answers) or len(contexts)!= len(questions):
        print("Hay un error en el tamaño de los arreglos")
        return
    test_size = len(contexts)
    em_average = 0
    f1_average = 0
    for n in range(test_size):
        #print(n)
        prediction = get_prediction(n, questions, contexts)
        print("Prediction: ",prediction)
        answer = answers[n]["text"]
        print("Answer: ",answer)
        em_score = compute_exact_match(prediction, answer)
        f1_score = compute_f1(prediction, answer)
        print(f"EM: {em_score} \t F1: {f1_score}")
        em_average += em_score
        f1_average += f1_score
    em_average_tot = em_average/test_size
    f1_average_tot = f1_average/test_size
    return em_average_tot, f1_average_tot

In [175]:
em_average, em_average = average_metrics( test_contexts, test_answers, test_questions)
print(f"Average EM: {em_average} \nAverage F1: {em_average}")


Prediction:  bone formation
Answer:  bonerestricted IFITM-like (BRIL) protein
EM: 0 	 F1: 0
Prediction:  HCoV‐HKU1
Answer:  HCoV‐HKU1
EM: 1 	 F1: 1.0
Prediction:  RNAi drug particles
Answer:  delivered to their site of action, be stable, enter the target cells, and be present in the cytoplasm at sufficient concentration
EM: 0 	 F1: 0
Prediction:  35 000 to 60 000
Answer:  35 000 to 60 000
EM: 1 	 F1: 0.8000000000000002
Prediction:  22.1
Answer:  22.1
EM: 1 	 F1: 1.0
Prediction:  Transmissibility and severity
Answer:  Transmissibility and severity
EM: 1 	 F1: 1.0
Prediction:  37uC overnight
Answer:  32uC, 37uC, 42uC
EM: 0 	 F1: 0.4
Prediction:  biotinylated
Answer:  two-dimensional gel electrophoresis , protein chip, mass spectrometry, and suspension array technology
EM: 0 	 F1: 0
Prediction:  denaturation step at 95°C for 10 min
Answer:  denaturation step
EM: 0 	 F1: 0.4444444444444445
Prediction:  to detect TB cases and the cost effectiveness of such intervention
Answer:  estimate the